# k-Armed Bandit

- The *value* is the *expected reward*
  $$
  \begin{align*}
  q_\star(a) \doteq& \mathbb{E}[R_t|A_t=a]\quad\forall a\in\{1,\cdots,k\} \\
                  =& \sum_r{p(r|a)r}
  \end{align*}
  $$
- The goal is to *maximize* the *expected reward*
  $$
  \underset{a}{\mathrm{argmax}}\ q_\star(a)
  $$

## Value of an Action

- $q_\star(a)$ is not known, so we *estimate* it

### Sample-Average Method

$$
\begin{align*}
Q_t(a) \doteq& \frac{\text{sum of rewards when $a$ taken prior to $t$}}{\text{# of times $a$ taken prior to $t$}} \\
            =& \frac{\sum^{t-1}_{i=1}{R_i}}{t-1}
\end{align*}
$$

### Incremental update

$$
\begin{align*}
Q_{n_1} =& \frac{1}{n}\sum^{n}_{i=1}{R_i} \\
        =& \frac{1}{n}(R_n + \sum^{n-1}_{i=1}{R_i}) \\
        =& \frac{1}{n}(R_n + (n - 1)\frac{1}{n-1}\sum^{n-1}_{i=1}{R_i}) \\
        =& \frac{1}{n}(R_n + nQ_n - Q_n) \\
        =& Q_n + \frac{1}{n}(R_n - Q_n)
\end{align*}
$$

### Incremental update rule

- This is an example of *Non-stationary bandit problem*

$$
\mathit{NewEstimate} \leftarrow \mathit{OldEstimate} + \mathit{StepSize}(\mathit{Target} - \mathit{OldEstimate})
$$
$$
Q_{n+1} = Q_n + \alpha_n(R_n - Q_n) \\
\alpha_n \rightarrow [0,1]
$$

### Decaying past rewards

- Inital action-value decreases exponentially with time
- The rewards further back in contribute exponentially less to the sum

$$
\begin{align*}
Q_{n+1} =& Q_n + \alpha_n(R_n - Q_n) \\
        =& \alpha R_n + (1-\alpha)\alpha R_{n-1} + (1-\alpha)^2 R_{n-2} + \cdots + (1-\alpha)^{n-1}\alpha R_1 + (1-\alpha)^nQ_1 \\
        =& (1-\alpha)^n\underbrace{Q_1}_{\text{initial action-value}} + \sum^{n}_{i=1}{\alpha(1-\alpha)^{n-i}R_i}
\end{align*}
$$